# 09B Fundal Flow (Beginner's Template)

## Preamble

Hi! Welcome to the CIGMAH Puzzle Hunt. If this is your first time programming in general, then we'd recommend you familiarise yourself with the basics of the Python language.

[Here's a tutorial from the official Python website.](https://docs.python.org/3/tutorial/introduction.html) It's long, but you should end up with a good basis for understanding the language if you work through it. If you'd like to just jump in to the notebook - sure! We'll try to explain things as best as we can, but you might struggle at first if you're completely new to programming. There are some great beginner's resources out there that can lay down foundations which are necessary no matter what domain you apply coding in. We're in the process of gathering some more beginner-friendly resources so hopefully we can provide some direction if you're completely lost - let us know!

For those of you new to Jupyter notebooks - these notebooks are a way to mix some text with code that we can run. If a cell has a grey background (and you're in the Jupyter environment!), then you can probably run it - click inside it and press `Shift Enter`. Try it!

In [ ]:
print("Hello, world!")

You should see `Hello, world!` printed below the code cell.

If you're on Azure, you will need to login and clone this notebook. If you're on Binder, you should be able to run the notebook as is!

## Problem Summary

We'll work through Problem 09B, *Fundal Flow*. 

[Here's a link to the puzzle.](https://cigmah.github.io/cgmnt/#/puzzles/9)

Have a look at the Puzzle Video: if you're viewing this notebook in the browser, you can click the following image to follow an external link to the video directly.

[![Fundal Flow](https://img.youtube.com/vi/843A9YynPyg/0.jpg)](https://www.youtube.com/watch?v=843A9YynPyg)

The problem in this puzzle is to essentially **distinguish funduscopy images between "normal" and "with a central retinal artery occlusion (CRAO)**. However, we aren't explicitly told what features distinguish these images (though you might be able to guess). We've therefore been given some **training** images of normal and CRAO funduscopies. Instead of explicitly telling the computer what to look for, we'll **train** the computer to discover what features best distinguish the training data, and then apply this to the **test** images.

This is a special dataset we've generated ourselves, and it's much more exaggerated than real life to help make this puzzle solvable easily. However, the principle for real-life data is similar - instead of *telling* the computer how to analyse the pictures, the computer will *learn* the best way to analyse them in order to achieve a goal. As you might guess, that's *machine learning*.

Machine learning has all sorts of applications in medicine. This is a basic example - distinguishing images. But you can imagine it could equally be applied, for example, to diagnostics, optimising therapies and re/sub-classifying diseases.

For this puzzle, we'll use [Keras](https://www.tensorflow.org/tutorials/keras/basic_classification), which is a simple interface to building neural networks with [TensorFlow](https://www.tensorflow.org/) under the hood. Using neural networks for this puzzle is definitely not strictly *necessary* (other methods, such as [k-means](https://en.wikipedia.org/wiki/K-means_clustering) or [SVM](https://en.wikipedia.org/wiki/Support-vector_machine) around basic image statistics are alternatives), but Keras makes this so easy to do that we'll use it anyway. This is a Beginner's puzzle, so we don't expect you to come up with any complicated neural networks - something "out-of-the-box" is more than sufficient. The main aim of this puzzle is to get you started!

To use Keras, we need to:

1. **Preprocess our training data into an [array](https://en.wikipedia.org/wiki/Array_data_structure) of images and an array of labels.**. You can imagine the array of images as a big, ordered list of 1000 images (though in truth, this is a `1000 x 75 x 75 x 3` array, but the `75 x 75 x 3` part just represents the image pixels - the image dimensions are `75 x 75 x 3`. The `3` at the end corresponds to [RGB](https://en.wikipedia.org/wiki/RGB_color_model) - every pixel in the 75x75 image contains a number representing the proportion of red, green and blue that pixel is made up of). The array of labels is just a list of 1000 labels in the same order as the images. 
2. **Create a model for our neural network.** We'll get you to do this with some guidance!
3. **Train the model on our preprocessed data.** This is just a few lines, Keras makes it very easy.
4. **Preprocess our test data in an array, just like our training data.** It's very important that our training and test data are processed in exactly the same way, otherwise our model will get confused since it's expecting somethng else!
5. **Make predictions for our test data.** Again, Keras makes this very easy.

Afterwards,

6. **We'll then extract all the letters of the images which were classified as "CRAO".** That'll give us the a riddle to solve, and we're done!

<br>

<center><h3>For this Beginner's Template, we're only going to get you to work on Step 2, making the neural network model.</h3></center>

<br>

This'll keep it short, and you don't have to worry about the (quite boring) preprocessing steps at this stage. Preprocessing will vary for whatever data you have, so we don't think it's worth your time to try doing it specifically here. We'll briefly describe what the steps involve, and if you'd like more help on learning how to do the preprocessing yourself then let us know! 

# Step 1: Preprocessing

Here, we'll preprocess this data. Don't worry too much about the code here at this stage.

<br>

<center><h3> Make sure to press `Shift-Enter` in the grey code cells before moving on - these cells need to run to make the data available to you!</h3></center>

<br>

If you're running this on Azure Notebooks, you need to install the required packages first. Uncomment the following lines (highlight the code and press `Ctrl-/`) and press `Shift-Enter` to run it. If you're running this outside, Azure, you'll need to ensure you have these packages installed (which should be as easy as running `pip install numpy pandas keras imageio` from the command line - let us know if you're having any issues and we can point you in the right direction!)

In [ ]:
# !pip install numpy pandas keras imageio

First we'll import the modules that we want to use. 

1. `re` - A library for [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) that'll help us process the text of the filenames. Check out [Puzzle 3B Combo Catcher](https://cigmah.github.io/cgmnt/#/puzzles/3) and [Puzzle 4C Troubled Test](https://cigmah.github.io/cgmnt/#/puzzles/4) for more practise on text processing.
2. `numpy` - A library for array processing.
3. `keras` - Our main interest!
4. `tensorflow` - The workhorse for keras.
5. `glob` - A library for quickly getting a list of files which fulfil a certain filename criteria, a bit like regular expressions.
6. `pandas` - A library for "data-frame" processing. See [Puzzle 4C Troubled Test](https://cigmah.github.io/cgmnt/#/puzzles/4) and [Puzzle 6B While You're Waiting](https://cigmah.github.io/cgmnt/#/puzzles/6) for some more practise with pandas.
7. `imageio` - A library for reading image pixel data from the image files.

In [ ]:
import re
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from glob import glob
from pandas import DataFrame as DF
from imageio import imread

Next, we'll actually run the preprocessing. *Don't worry about these lines for now.* We've written it in quite a dense style for brevity, but just make sure to hit `Shift-Enter` inside the grey cell to run it. Depending on where you're running this, it might take a little bit of time to complete while it's reading the 1000 training images.

In [ ]:
trainglob = "./inputs/input09_training/*.png"
trainfs = sorted(glob(trainglob))
renm = re.compile(r".*?(\d+)_(\w+).png")
trainlabs = DF.from_records([{k: renm.match(f)[i] for k,i in zip(["id", "class"], [1,2])} for f in trainfs])
trainlabs["class"] = trainlabs["class"].apply(lambda c: 0 if c=="normal" else 1)
trainims = np.stack([imread(f) / 255 for f in trainfs], axis=0)

If you're interested, here's what each line does:

1. The first line defines the `glob` for finding all the image files in the training folder.
2. The second line runs the `glob` to generate a list of file paths, and sorts them in order. 
3. The third line defines a regular expression that will parse the names of the image files into the digit ID, and the word that comes after it (for this set, "normal" or "crao" - their class).
4. The fourth line creates a dataframe (like an Excel spreadsheet) of all the image IDs and their class. 
5. The fifth line assigns all the class labels a number (0 for normal, or 1 for CRAO).
6. The sixth line reads all the image files, scales them to values between 0 and 1 by dividing the RGB values by 255, and "stacks" them in the 0-th axis (effectively creating a `1000 x image` array.

And that's the preprocessing step done.

# Step 2: Creating a Model and Step 3: Training the Model

Here's where the fun starts!

You can imagine a neural network as taking an array of data and processing that data with multiple different layers which output a new array of data. Eventually, you reach a layer which just outputs a 2-element array (for this puzzle) - 1 element for each class (normal or CRA0). Here's an image from Wikipedia:

![ANN](https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/300px-Colored_neural_network.svg.png)

Progressive layers of the network interpret more abstracted features and connect with layers before and after it - for example, one layer might just be examining raw pixels to find edges of a certain orientation, and a layer after it might be using those edge orientations to find circles, and a layer after that might be using those circles to find optic discs...you get the picture. 

We're going to refer you to the [basic classification tutorial section on setting up layers](https://www.tensorflow.org/tutorials/keras/basic_classification#setup_the_layers) to have a look at how this is done. Your job is to *apply* this to this data set - just replace the question mark with layers. You can follow the official Keras tutorial fairly closely, and get something with fairly reasonable accuracy - experiment with different combinations of layers, different numbers of neurons in a layer, different activation functions etc. 

We'd suggest you do some more reading on neural networks (unfortunately, we're a bit short of time to do it here), but to give you a bare minimum, consider this layer:

```python
keras.layers.Dense(128, activation=tf.nn.relu)
```

This layer has 128 neurons that are densely connected (fully-connected) to all the the neurons or pixels in the layer before it. Each neuron takes all the neurons/pixels in the layer before it, multiplies the values of those neurons/pixels with variable *weights* and adds a *bias* value, adds those values together, runs it through the activation function (ReLU, which just converts any negative numbers to 0 and a positive number to itself), then outputs that value which will feed into the next layer. When we train the network, we are *optimising* the weights and bias values that produce the best capability of the network to distinguish the images. 

The example in the Keras tutorial of a model goes like this:

```python
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
```

Their image sizes are different (as is the final number of classes), so bear that in mind. If you just slot it in below, you'll get an error when you try training the model because the image dimensions aren't right. You need to modify it to suit this dataset. Make sure to remember that your input shape is `(75,75,3)` (i.e. each image is a `75 x 75 x 3` array). You'll need this for the very first layer, which flattens the data out into a `16,875` single dimension array. Also make sure to remember that your output (final) layer should only have `2` neurons, since we only have two classes.

Keras hides all the maths, but if you're interested, [Deep Learning Book](http://www.deeplearningbook.org/) is a freely-available and thorough introduction to more advanced concepts.




In [ ]:
# Replace the question mark below with your layers!
model = keras.Sequential([
    ?
])

We then do Step 3, which is training the model. Don't worry too much about the particulars of this at the moment, though you can modify it if you'd like - the compiled model parameters are directly from the Keras tutorial. Hit `Shift-Enter` and see your model train itself.

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(trainims, np.array(trainlabs["class"]), epochs=5)

You should see the model training every epoch. If you get an error, something's wrong with your layers.

Have a look at the accuracy (`acc`) at each epoch - does it get closer to 1? If not, try rejigging those layers - add more layers, change the number of neurons. We can assure you that you won't have to make too many changes to get close to 100% accuracy.

# Step 4: Preprocessing The Test Data

This is exactly like Step 1. We won't discuss it since it's almost exactly the same, just make sure to run `Shift-Enter` in the grey cell! Again, it might take a little time to run while it reads all those images.

Importantly, the `testlabs` variable defined in the final line extracts all the letter labels of each image and puts it into a dataframe with each image ID. We'll use this to extract the riddle at the end.

In [ ]:
testglob = "./inputs/input09_test/*.png"
testfs = sorted(glob(testglob))
testims = np.stack([imread(file) / 255 for file in testfs], axis=0)
testlabs = DF.from_records([{k: renm.match(f)[i] for k,i in zip(["id", "class"], [1,2])} for f in testfs])

# Step 5: Making Predictions for the Test Data

Now that we've preprocessed the test data, Keras makes predicting the classifications of the test images a one-liner.

In [ ]:
preds = [np.argmax(p) for p in model.predict(testims)]

Essentially, this allocates a "confidence" to each class for each image using `model.predict()`. `np.argmax()` then finds which class had the highest confidence.

# Step 6: Extracting the Riddle

Finally, we extract the riddle. We basically just extract all the images where the prediction was 1 (i.e. CRAO) and extract the letter labels from the dataset for each of those images. We then print those letters out in order below. Hit `Shift-Enter` in the code cell below.

In [ ]:
icrao = np.array([i for i, p in enumerate(preds) if p == 1])
letters = "".join(testlabs.loc[icrao,"class"])
print(letters)

Did you get somethng recognisable? There aren't any spaces, but you should be able to read it (plus or minus a few letters, since the classification accuracy is almost never going to be perfect.)

If you did, hooray! Answer the riddle and submit it as your answer. Otherwise, you'll have to rejig those layers again to improve your classification accuracy!